In [1]:
# -*- coding: UTF-8 -*-
import numpy as np
import pandas as pd
import jieba
import jieba.analyse
import codecs

# Seperate words for the text

In [2]:
from collections import Counter
import jieba
jieba.load_userdict('userdict.txt')

# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r').readlines()]
    return stopwords

# 对句子进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist('stoped.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr

inputs = open('哈利波特与魔法石(人文版).txt', 'r',encoding="utf-8") #加载要处理的文件的路径
outputs = open('output.txt', 'w') #加载处理后的文件路径
for line in inputs:
    line_seg = seg_sentence(line)  # 这里的返回值是字符串
    outputs.write(line_seg)
outputs.close()
inputs.close()
# WordCount
with open('output.txt', 'r') as fr: #读入已经去除停用词的文件
    data = jieba.cut(fr.read())
data = dict(Counter(data))

with open('cipin.txt', 'w') as fw: #读入存储wordcount的文件路径
    for k, v in data.items():
        fw.write('%s,%d\n' % (k, v))

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/89/qp8kb90s49z7xcdskq2c0qlm0000gn/T/jieba.cache
Loading model cost 0.872 seconds.
Prefix dict has been built succesfully.


In [3]:
# 拆分句子
# 版本为python3，如果为python2需要在字符串前面加上u
import re
def cut_sent(para):
    para = re.sub('([。！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('([，！？\?])([^”’])', r"\1\n\2", para)  # 单字符断句符
    para = re.sub('(\.{6})([^”’])', r"\1\n\2", para)  # 英文省略号
    para = re.sub('(\…{2})([^”’])', r"\1\n\2", para)  # 中文省略号
    para = re.sub('([。！？\?][”’])([^，。！？\?])', r'\1\n\2', para)
    
    # 如果双引号前有终止符，那么双引号才是句子的终点，把分句符\n放到双引号后，注意前面的几句都小心保留了双引号
    para = para.rstrip()  # 段尾如果有多余的\n就去掉它
    # 很多规则中会考虑分号;，但是这里我把它忽略不计，破折号、英文双引号等同样忽略，需要的再做些简单调整即可。
    return para.split("\n")


# Calculate the length of the sentenct

In [4]:
dict_sentence_len=dict()
inputs = open('哈利波特与魔法石(人文版).txt', 'r',encoding="utf-8") #加载要处理的文件的路径
for line in inputs:
    l=cut_sent(line)
    for w in l:
        line_seg = seg_sentence(w) 
        words=list(line_seg.split(' '))
        dict_sentence_len[line_seg]=len(words)-1
        #print(words)
        #print(len(words))

# Transform the dict into dataframe and save the results of sentence-length into .xls file

In [5]:
df_sentence_len=pd.DataFrame(columns = ['句子','长度'])
df_sentence_len.head()
df_sentence_len['句子']=dict_sentence_len.keys()
df_sentence_len['长度']=dict_sentence_len.values()
df_sentence_len.to_excel('句子长度.xls')

# Read character file

In [6]:
text = open('哈利波特与魔法石(人文版).txt', 'r',encoding="utf-8") #加载要处理的文件的路径
roles_df=pd.read_excel('character.xlsx',names=['character'],index=[0])
roles_df.head()

,character
0,邓布利多
1,亚瑟
2,韦斯莱
3,巴特缪斯
4,巴蒂


# Calculate the frequency of roles in certain length text

In [7]:
def calculate_roles(inputs):
    roles_dict=dict()
    
    for line in inputs:
        line_seg = seg_sentence(line)
        words_list=list(line_seg.split(' '))
        #print(words_list)
        for role in roles_df['character']:
            #print(role)
            if role in words_list:
                if role not in roles_dict:
                    roles_dict[role]=1
                else:
                    roles_dict[role]+=1
                #print(role)
    return roles_dict
    

In [9]:
# Calculate roles of the whole text
#calculate_roles(text)

# Read raw text

In [35]:
text = open('哈利波特与魔法石(人文版).txt', 'r',encoding="utf-8") #加载要处理的文件的路径
chapter_df=pd.read_excel('chapters.xlsx',names=['chapter','','',''],index=[0])
chapter_df.head()

,chapter,,,
0,第2章,NaN,NaN,NaN
1,第3章,NaN,NaN,NaN
2,第4章,NaN,NaN,NaN
3,第5章,NaN,NaN,NaN
4,第6章,NaN,NaN,NaN


In [11]:
# Seperate

# seperate chapters and save all chapters into one list

In [36]:
i=0
one_chapter=list() #used to save all lines in single chapter
all_chapters=list() # used to save all chapters into one list
for line in text:        # read all lines
    if chapter_df['chapter'][i] in line: # identify the chapter mark
        #print(chapter_df['chapter'][i])
        all_chapters.append(one_chapter) # if true, we save the single chapter into the chapter list
        one_chapter=list() # then we initialize the chapter list
        i+=1
    one_chapter.append(line)      

# calculate character frequency in each chapter

In [37]:
for chaps in all_chapters:  
    print(calculate_roles(chaps))

{'哈利': 24, '海格': 11, '赫敏': 1, '纳威': 1, '马尔福': 2, '伏地魔': 5, '波特': 18, '闪闪': 1, '邓布利多': 30, '庞弗雷夫人': 1, '小天狼星布莱克': 1}
{'哈利': 46}
{'哈利': 44, '波特': 2}
{'哈利': 38, '海格': 31, '邓布利多': 8, '波特': 8, '伏地魔': 1}
{'哈利': 91, '海格': 70, '邓布利多': 4, '波特': 18, '汤姆': 1}
{'哈利': 72, '海格': 10, '纳威': 6, '韦斯莱': 5, '伏地魔': 2, '邓布利多': 3, '赫敏': 8, '格兰杰': 3, '德拉科': 2, '马尔福': 14, '高尔': 4, '波特': 4}
{'哈利': 41, '海格': 4, '纳威': 6, '赫敏': 5, '皮皮鬼': 5, '格兰杰': 1, '马尔福': 8, '高尔': 1, '波特': 4, '韦斯莱': 5, '邓布利多': 9, '差点没头的尼克': 3, '血人巴罗': 5, '费尔奇': 1}
{'哈利': 32, '差点没头的尼克': 1, '皮皮鬼': 2, '费尔奇': 4, '韦斯莱': 3, '宾斯教授': 1, '赫敏': 7, '格兰杰': 2, '海格': 14, '德拉科': 1, '马尔福': 6, '波特': 12, '高尔': 1, '纳威': 3}
{'德拉科': 2, '马尔福': 64, '哈利': 62, '纳威': 18, '赫敏': 16, '格兰杰': 5, '海格': 3, '高尔': 4, '韦斯莱': 4, '波特': 14, '邓布利多': 2, '皮皮鬼': 12, '费尔奇': 13, '胖夫人': 5, '庞弗雷夫人': 1, '血人巴罗': 1}
{'马尔福': 10, '哈利': 62, '赫敏': 22, '纳威': 3, '高尔': 1, '波特': 6, '韦斯莱': 4, '格兰杰': 8, '邓布利多': 3, '皮皮鬼': 1, '血人巴罗': 1, '胖夫人': 1}
{'海格': 16, '哈利': 46, '赫敏': 21, '波特': 8, '费尔奇': 2, '邓布利多': 3,

In [39]:
calculate_roles(chaps)

{'哈利': 90,
 '伏地魔': 5,
 '纳威': 12,
 '赫敏': 59,
 '宾斯教授': 1,
 '韦斯莱': 2,
 '庞弗雷夫人': 1,
 '邓布利多': 12,
 '海格': 12,
 '波特': 4,
 '胖夫人': 1,
 '费尔奇': 3,
 '皮皮鬼': 6,
 '血人巴罗': 1,
 '斯普劳特教授': 1}

In [15]:
df_sentence_len=pd.DataFrame(columns = ['角色','出现次数'])
df_sentence_len.head()
df_sentence_len['句子']=dict_sentence_len.keys()
df_sentence_len['长度']=dict_sentence_len.values()
df_sentence_len.to_excel('句子长度.xls')

In [16]:
chapter_df=pd.read_excel('chapters.xlsx',names=['chapter','','',''],index=[0])
chapter_df.head()

,chapter,,,
0,第2章,NaN,NaN,NaN
1,第3章,NaN,NaN,NaN
2,第4章,NaN,NaN,NaN
3,第5章,NaN,NaN,NaN
4,第6章,NaN,NaN,NaN
